In [1]:
# Imports
import re
import string
from collections import Counter

import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup

In [2]:
# Load the raw data - data files are split in two due to their size
df1 = pd.read_csv('https://raw.githubusercontent.com/JimKing100/techsearch/master/data/techsearch_p1.csv')
df1 = df1.drop(df1.columns[0], axis=1)
df2 = pd.read_csv('https://raw.githubusercontent.com/JimKing100/techsearch/master/data/techsearch_p2.csv')
df2 = df2.drop(df2.columns[0], axis=1)
df = pd.concat([df1, df2], ignore_index=True)

In [3]:
# Clean the text
def clean_text(text):
    text = text.replace('\n', ' ')                # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

df['description'] = df.apply(lambda x: clean_text(x['description']), axis=1)

In [4]:
# Initialize the tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)
STOP_WORDS = nlp.Defaults.stop_words.union(['year'])

In [5]:
# Tokenizer pipe removing stop words and blank words and lemmatizing
tokens = []

for doc in tokenizer.pipe(df['description'], batch_size=500):
    
    doc_tokens = []
    for token in doc:
        if (token.lemma_ not in STOP_WORDS) & (token.text != ' '):
            doc_tokens.append(token.lemma_)

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [6]:
df.head()

,job_title,company,location,description,counts,city,job,low_salary,high_salary,tokens
0,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",help build technology that saves lives were a...,1259,San Jose,data scientist,NaN,NaN,"[help, build, technology, save, live, fast, gr..."
1,Data Scientist (Intern) - United States,Cisco Careers,"San Jose, CA",what youll doacquire clean and structure data ...,1259,San Jose,data scientist,NaN,NaN,"[youll, doacquire, clean, structure, datum, mu..."
2,Data Scientist,Stanford University,"Stanford, CA",data scientist data analyst 2 job family infor...,1259,San Jose,data scientist,NaN,NaN,"[datum, scientist, datum, analyst, 2, job, fam..."
3,"Data Scientist in Santa Clara, CA (corp-corp c...",Advantine Technologies,"Santa Clara, CA",job description title data scientist locatio...,1259,San Jose,data scientist,NaN,NaN,"[job, description, title, datum, scientist, lo..."
4,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008",job title data scientistlocation campbell ca 9...,1259,San Jose,data scientist,150000.0,210000.0,"[job, title, datum, scientistlocation, campbel..."


In [7]:
# Tech terms list
tech_terms = ['python', 'r', 'sql', 'hadoop', 'spark', 'java', 'sas', 'tableau',
              'hive', 'scala', 'aws', 'c', 'c++', 'matlab', 'tensorflow', 'excel',
              'nosql', 'linux', 'azure', 'scikit', 'machine learning', 'statistic',
              'analysis', 'computer science', 'visual', 'ai', 'deep learning',
              'nlp', 'natural language processing', 'neural network', 'mathematic',
              'database', 'oop', 'blockchain',
              'html', 'css', 'javascript', 'jquery', 'git', 'photoshop', 'illustrator',
              'word press', 'seo', 'responsive design', 'php', 'mobile', 'design', 'react',
              'security', 'ruby', 'fireworks', 'json', 'node', 'express', 'redux', 'ajax',
              'java', 'api', 'state management',
              'wireframe', 'ui prototype', 'ux writing', 'interactive design',
              'metric', 'analytic', 'ux research', 'empathy', 'collaborate', 'mockup', 
              'prototype', 'test', 'ideate', 'usability', 'high-fidelity design',
              'framework',
              'swift', 'xcode', 'spatial reasoning', 'human interface', 'core data',
              'grand central', 'network', 'objective-c', 'foundation', 'uikit', 
              'cocoatouch', 'spritekit', 'scenekit', 'opengl', 'metal', 'api', 'iot',
              'karma']

In [8]:
df['tokens_filtered'] = df.apply(lambda x: list(set(x['tokens']) & set(tech_terms)), axis=1)

In [9]:
df.head()

,job_title,company,location,description,counts,city,job,low_salary,high_salary,tokens,tokens_filtered
0,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",help build technology that saves lives were a...,1259,San Jose,data scientist,NaN,NaN,"[help, build, technology, save, live, fast, gr...","[test, sql, analysis, python, analytic, design]"
1,Data Scientist (Intern) - United States,Cisco Careers,"San Jose, CA",what youll doacquire clean and structure data ...,1259,San Jose,data scientist,NaN,NaN,"[youll, doacquire, clean, structure, datum, mu...","[javascript, database, excel, hadoop, metric, ..."
2,Data Scientist,Stanford University,"Stanford, CA",data scientist data analyst 2 job family infor...,1259,San Jose,data scientist,NaN,NaN,"[datum, scientist, datum, analyst, 2, job, fam...","[database, metric, statistic, prototype, test,..."
3,"Data Scientist in Santa Clara, CA (corp-corp c...",Advantine Technologies,"Santa Clara, CA",job description title data scientist locatio...,1259,San Jose,data scientist,NaN,NaN,"[job, description, title, datum, scientist, lo...","[test, nlp, analysis, python, analytic]"
4,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008",job title data scientistlocation campbell ca 9...,1259,San Jose,data scientist,150000.0,210000.0,"[job, title, datum, scientistlocation, campbel...","[network, python]"


In [10]:
# Create a count function
def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(temp, columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc.sort_values(by='rank')

In [11]:
# Load the final dataframe with the approriate data
def populate_df(title, city):
    j_title = df['job'] == title
    j_city = df['city'] == city
    subset_df = df[j_title & j_city]
    subset_df = subset_df.reset_index()
    
    wc = count(subset_df['tokens_filtered'])
    skills = wc['word'][:10]
    
    if subset_df.shape[0] > 0:
        data = {'job': title,
                'city': city,
                'counts': subset_df['counts'][0],
                'low_salary': subset_df['low_salary'].mean(),
                'high_salary': subset_df['high_salary'].mean(),
                'skills': list(skills)}
    else:
        data = {'job': title,
                'city': city,
                'counts': 0,
                'low_salary': 0,
                'high_salary': 0,
                'skills': []}
    
    return data
    

In [12]:
final_df = pd.DataFrame(columns=['job', 'city', 'counts', 'low_salary', 'high_salary', 'skills'])

results = populate_df('data scientist', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [13]:
results = populate_df('web developer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [14]:
results = populate_df('ux designer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [15]:
results = populate_df('ios developer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [16]:
# Clean up the salary data
final_df['low_salary'] = final_df['low_salary'].fillna(0)
final_df['low_salary'] = final_df['low_salary'].apply(lambda x: int(x))
final_df['low_salary'] = final_df['low_salary'].apply(lambda x: 0 if x < 10000 else x)
final_df['high_salary'] = final_df['high_salary'].fillna(0)
final_df['high_salary'] = final_df['high_salary'].apply(lambda x: int(x))
final_df['high_salary'] = final_df['high_salary'].apply(lambda x: 0 if x < 10000 else x)

In [17]:
final_df.head()

,job,city,counts,low_salary,high_salary,skills
0,data scientist,San Jose,1259,83333,150000,"[python, analysis, statistic, design, r, sql, ..."
1,data scientist,San Francisco,912,100000,150000,"[python, analysis, statistic, design, r, sql, ..."
2,data scientist,Seattle,1075,109629,153364,"[analysis, python, statistic, sql, design, r, ..."
3,data scientist,Washington,1161,72796,106703,"[analysis, statistic, python, analytic, design..."
4,data scientist,New York,820,63502,81611,"[python, analysis, sql, r, design, statistic, ..."


In [18]:
final_df.to_csv('scrape_results.csv')